In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotnine import *



In [11]:

lngSet = pd.read_pickle("tidySets.pkl")
#lngSet.to_csv("longSets.csv")

In [3]:
df = lngSet
import re

def extractGrade(s):
    match = re.search(r'(\d{1,2})\D*$', s)
    return int(match.group(1)) if match else 0

# Test cases
#print(extract_last_two_numbers("MINT9"))  # Output: 9
#print(extract_last_two_numbers("Auth"))   # Output: 0

df["Grade"] = df['Grade'].apply(lambda x: extractGrade(x))
df["Grade"] = pd.to_numeric(df["Grade"])

df["Year_Diff"] = 2023 - df["Year"].dt.year


df2 = (
    df
    .loc[df['Year'].dt.year > 1959]
    .groupby(["Set Name", "Year", "Year_Diff"])
    .agg(
        Total_Pop=('Count', 'sum'),
        Gems=('Count', lambda x: x[df['Grade'] == 10].sum())
    )
    .reset_index()
)


In [4]:
df2 = (
    df
    .loc[df['Year'].dt.year > 1900]
    .groupby(["Set Name", "Year", "Year_Diff"])
    .agg(
        Total_Pop=('Count', 'sum'),
        Gems=('Count', lambda x: x[df['Grade'] == 10].sum())
    )
    .reset_index()
)

df2["Gem_MT_%"] = df2["Gems"]/ df2["Total_Pop"]
df2 = df2.loc[df2["Total_Pop"] > 500]

In [23]:
lngSet = pd.read_pickle("tidySets.pkl")
# Assuming lngSet is your DataFrame
df = lngSet
df["Year"] = df["Year"].dt.year
df["Set Name"] = df["Year"].astype(str) + " " + df["Set Name"]

def extractGrade(s):
    match = re.search(r'(\d{1,2})\D*$', s)
    return int(match.group(1)) if match else 0

df["Grade"] = df['Grade'].apply(lambda x: extractGrade(x))
df["Grade"] = pd.to_numeric(df["Grade"])

df3 = (
    df
    .groupby(["Set Name", "Grade"])
    .agg(
        Total_Pop=('Count', 'sum')
    )
    .reset_index()
)

df3["avg"] = df3["Grade"] * df3["Total_Pop"]

# Corrected aggregation
df3 = (
    df3
    .groupby(["Set Name", "Year"])
    .agg(
        Mean=("avg", "sum"),
        Total_Pop=("Total_Pop", "sum")
    )
    .assign(Mean=lambda x: x["Mean"] / x["Total_Pop"])
    .reset_index()
)

# Optionally, you can round the mean to a specific number of decimal places
# df3["Mean"] = df3["Mean"].round(2)


In [ ]:
gemScatterPlot = (ggplot(df2,mapping = aes(x = "Year_Diff", y = "Gem_MT_%")) +
    geom_point() +
    labs(
        title = "Gem Mint% vs Years Ago from 2023",
        x = "Years Ago from 2023",
        y = "Gem Mint%",
        subtitle = "Min Pop 500"
    ) +
    theme_bw()
    
    
    )

gemScatterPlot

In [ ]:
topSetPop2 = (
  lngSet
  .loc[lngSet['Year'].dt.year > 1999]
  .groupby(["Set Name", "Year"])
  .agg(
    Total_Pop = ("Count", "sum")
    
  )
  .reset_index()
  .sort_values(by="Total_Pop", ascending=False) 
  .head(10)
  
)

topSetPop2["Year"] = topSetPop2["Year"].dt.year
topSetPop2["Set Name"] = topSetPop2["Year"].astype(str) + " " + topSetPop2["Set Name"]

x = topSetPop2




In [ ]:

recentCards = (ggplot(x, mapping = aes(x = "Set Name", y = "Total_Pop")) +
            geom_col(fill = "lightblue", color = "blue", size = .5) +
            coord_flip() +
            scale_x_discrete(limits=topSetPop2.sort_values('Total_Pop', ascending=True)['Set Name']) +
            theme_bw()+
            labs(
                title = "Total Number of Cards Graded for the Top 10 Sets since 2000",
                x ="",
                y = "Number of Cards Graded"
            )
        

)

In [ ]:
recentCards + theme(figure_size=(12,8))

In [ ]:
df3 = df2
df3["Year"] = df3["Year"].dt.year
df3["Set Name"] = topSetPop2["Year"].astype(str) + " " + topSetPop2["Set Name"]

In [ ]:
df.head
